In [18]:
import geopandas as gpd
import folium
import pandas as pd
from folium.plugins import LocateControl
from shapely.geometry import Point
from shapely.ops import cascaded_union

In [2]:
walled_cities_dict = {"Tiberias":[32.775011, 35.545584],"Hebron":[31.524723, 35.110460],"Nablus":[32.213117, 35.284881],
                      "Jaffa":[32.053705, 34.750625],"Lod":[31.955268, 34.896203],"Gaza":[31.504135, 34.464426],
                      "Zefat":[32.966292, 35.494927],"Acre":[32.920831, 35.069042],"Haifa":[32.800833, 35.019167],
                      "Beit Shean":[32.499474, 35.498450],"Jericho":[31.870487, 35.440522],
                      "Beer Sheva":[31.237182, 34.793101],"Ramla":[31.924759, 34.872939],"Tyre":[33.274876, 35.193785],
                      "Sidon":[33.562539, 35.369619],"Damascus":[33.511125, 36.301019],"Izmir":[38.418095, 27.139753],
                      "Baghdad":[33.350506, 44.401307],"Jerusalem":[31.777055, 35.234268]}
walled_cities_gdf = gpd.GeoDataFrame()
walled_cities_gdf['City'] = list(walled_cities_dict.keys())
walled_cities_gdf['geometry'] = list(walled_cities_dict.values())
walled_cities_gdf['geometry'] = walled_cities_gdf['geometry'].apply(lambda x: Point(x[1],x[0]))
walled_cities_gdf.crs = {'init':'epsg:4326'}
walled_cities_gdf = walled_cities_gdf.to_crs(epsg=3857)

In [3]:
### File downloaded from https://download.geofabrik.de/asia/israel-and-palestine.html
buildings = gpd.read_file('/home/arieh/Downloads/israel-and-palestine-latest-free.shp/gis_osm_buildings_a_free_1.shp')
buildings = buildings.to_crs(epsg=3857)
buildings_cent = buildings.copy()
buildings_cent['geometry'] = buildings_cent['geometry'].centroid
buildings_cent['geometry'] = buildings_cent['geometry'].buffer(67.8)

In [4]:
all_union = cascaded_union(buildings_cent['geometry'])
all_union_polys = []
for poly in all_union:
    all_union_polys.append(poly)
max_buffers_gdf = gpd.GeoDataFrame()
max_buffers_gdf['geometry'] = all_union_polys
max_buffers_gdf['id'] = range(len(all_union_polys))
max_buffers_gdf.crs = buildings_cent.crs
shushan_extents = gpd.sjoin(walled_cities_gdf,max_buffers_gdf)
max_buffers_gdf_subset = max_buffers_gdf[max_buffers_gdf['id'].isin(shushan_extents['index_right'])]
max_buffers_gdf_subset = max_buffers_gdf_subset.to_crs({'init':'epsg:4326'})
Shushan_Zones = pd.merge(max_buffers_gdf_subset,shushan_extents[['id','City']],on = 'id')

In [22]:
x1,y1,x2,y2 = Shushan_Zones['geometry'].total_bounds
m = folium.Map(tiles='openstreetmap')
LocateControl().add_to(m)
m.fit_bounds([[y1, x1], [y2, x2]])
folium.GeoJson(Shushan_Zones,name='Shushan Zones').add_to(m)
folium.LayerControl().add_to(m)
m.save('local_files/Shushan_Zone_Tool.html')